# Libraries

In [1]:
!pip install scipy  

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse import csr_matrix

# Data

In [3]:
books = pd.read_csv("/kaggle/input/bookrecommendation/BX-Books.csv",sep=";",on_bad_lines='skip', encoding='latin-1')

/tmp/ipykernel_32/3767986735.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("/kaggle/input/bookrecommendation/BX-Books.csv",sep=";",on_bad_lines='skip', encoding='latin-1')


In [4]:
books.head(3)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...


In [5]:
books.shape

(271360, 8)

In [6]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [7]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher','Image-URL-L' ]]

### renaming columns

In [8]:
books.rename(columns={
    'Book-Title' : 'title', 
    'Book-Author' : 'author', 
    'Year-Of-Publication':'year', 
    'Publisher':'publisher',
    'Image-URL-L' : 'url'
}, inplace =True)


In [9]:
books.head(3)

,ISBN,title,author,year,publisher,url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...


## users data

In [10]:
users = pd.read_csv("/kaggle/input/bookrecommendation/BX-Users.csv",sep=";",on_bad_lines='skip', encoding='latin-1')

In [11]:
users.head(3)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN


In [12]:
users.isnull().sum()

User-ID          0
Location         0
Age         110762
dtype: int64

## Ratings data

In [13]:
ratings = pd.read_csv("/kaggle/input/bookrecommendation/BX-Book-Ratings.csv",sep=";",on_bad_lines='skip', encoding='latin-1')

In [14]:
ratings.head(3)

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0


In [15]:
ratings.shape

(1149780, 3)

In [16]:
ratings.isnull().sum()

User-ID        0
ISBN           0
Book-Rating    0
dtype: int64

In [17]:
ratings.rename(columns={
    "User-ID":"userid",
    'Book-Rating':'rating'
}, inplace = True)
ratings.head(3)

,userid,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0


## Only taking users who read more than 200 books

In [18]:
ratings.head(3)

,userid,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0


### rating means number of user read the books and rated it 

In [19]:
x = ratings['userid'].value_counts() > 100 # this shows the user who read over 100 books 

In [20]:
x[x].shape

(1825,)

In [21]:
  y = x[x].index # this is the index of user who read over 100 books

In [22]:
ratings = ratings[ratings['userid'].isin(y)] # this is the new data frame where the user read over 100 books

In [23]:
ratings.head(3)

,userid,ISBN,rating
412,276925,0006511929,0
413,276925,002542730X,10
414,276925,0060520507,0


In [24]:
ratings.shape

(656605, 3)

## Merging dataframe

In [25]:
ratings.head(3)

,userid,ISBN,rating
412,276925,0006511929,0
413,276925,002542730X,10
414,276925,0060520507,0


In [26]:
books.head(3)

,ISBN,title,author,year,publisher,url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...


In [27]:
ratings_with_books =  ratings.merge(books, on = "ISBN")
ratings_with_books.shape

(604854, 8)

In [28]:
ratings_with_books.head(3)

,userid,ISBN,rating,title,author,year,publisher,url
0,276925,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
2,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...


In [29]:
num_rating = ratings_with_books.groupby('title')['rating'].count().reset_index()

In [30]:
num_rating.head(3)

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,3
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1


In [31]:
num_rating.rename(columns= {
    "rating":'num_rating'
}, inplace = True)
num_rating.head(3)

,title,num_rating
0,A Light in the Storm: The Civil War Diary of ...,3
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1


## merging with the rating_of_bookds

In [32]:
ratings_with_books.columns

Index(['userid', 'ISBN', 'rating', 'title', 'author', 'year', 'publisher',
       'url'],
      dtype='object')

In [33]:
num_rating.columns

Index(['title', 'num_rating'], dtype='object')

In [34]:
final_rating =  ratings_with_books.merge(num_rating, on = 'title')

In [35]:
final_rating.head(3)

,userid,ISBN,rating,title,author,year,publisher,url,num_rating
0,276925,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,105
1,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,105
2,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,105


# Incase you get confused, we are building a collborative system so, we need different users also rating the same book 

## we are going to take books which has over 50 rating 

In [36]:
final_rating.shape

(604854, 9)

In [37]:
final_rating = final_rating[final_rating['num_rating']>=50]

In [38]:
final_rating.shape

(100426, 9)

In [39]:
final_rating.head(3)

,userid,ISBN,rating,title,author,year,publisher,url,num_rating
0,276925,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,105
1,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,105
2,3363,002542730X,0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,105


In [40]:
final_rating.drop_duplicates(['userid', 'title'], inplace = True)

In [41]:
final_rating.shape

(97962, 9)

# Dataframe ready

## Pivot table

In [42]:
book_pivot = final_rating.pivot(columns='userid', index = 'title', values ='rating')

In [43]:
book_pivot

userid,183,254,507,882,1424,1435,1733,1903,2033,2110,...,276018,276463,276680,276925,277427,277478,277639,278137,278188,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,NaN,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010: Odyssey Two,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
204 Rosewood Lane,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
24 Hours,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 1094 books, 1794 users

In [44]:
book_pivot.shape # 1094 books, 1794 users

(1094, 1794)

In [45]:
book_pivot.fillna(0, inplace = True)

In [46]:
book_pivot[0:10]

userid,183,254,507,882,1424,1435,1733,1903,2033,2110,...,276018,276463,276680,276925,277427,277478,277639,278137,278188,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010: Odyssey Two,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
204 Rosewood Lane,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
24 Hours,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A 2nd Helping of Chicken Soup for the Soul (Chicken Soup for the Soul Series (Paper)),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
book_sparse= csr_matrix(book_pivot) # from scipy.sparse -> we transformed it into a sparse matrix

# Clustering

In [48]:
from sklearn.neighbors import NearestNeighbors

In [50]:
model = NearestNeighbors(algorithm='brute')

In [51]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [52]:
distance , suggesstion  = model.kneighbors(
book_pivot.iloc[237,:].values.reshape(1,-1),
    n_neighbors = 6
) # 237 is the book of harry potter, we trying to find the books that are close to harry potter

In [53]:
distance

array([[ 0.        , 36.0970913 , 36.27671429, 36.61966685, 37.09447398,
        37.13488926]])

In [54]:
suggesstion

array([[ 237, 1020,  335,  620,  561,  654]])

In [57]:
for i in range(len(suggesstion)):
    print(book_pivot.index[suggesstion[i]])

Index(['Dr. Atkins' New Diet Revolution',
       'Tom Clancy's Op-Center: Games of State (Tom Clancy's Op Center (Paperback))',
       'Ground Zero and Beyond',
       'Ruthless.Com (Tom Clancy's Power Plays (Paperback))',
       'Personal Injuries', 'Shadows'],
      dtype='object', name='title')


In [58]:
books_name = book_pivot.index # saving the suggested books 

# saving the model 

In [59]:
import pickle

In [61]:
pickle.dump(model, open('model.pkl','wb'))

In [62]:
pickle.dump(books_name, open('books_name.pkl','wb'))
pickle.dump(final_rating, open('final_rating.pkl','wb'))
pickle.dump(book_pivot, open('book_pivot.pkl','wb'))

In [67]:
np.where(book_pivot.index == "2nd Chance")[0]

array([5])

In [69]:
np.where(book_pivot.index == "2nd Chance")[0][0] # returns the position of the book in the array. 

5

In [75]:
def recommend_book(book_name):
    book_id = np.where(book_pivot.index == books_name)[0][0]
    distance , suggesstion  = model.kneighbors(book_pivot.iloc[book_id,:].values.reshape(1,-1),n_neighbors = 6)
    for i in range (len(suggesstion)):
            books = book_pivot.index[suggesstion[i]]
            for j in books:
                print(j)

In [76]:
recommend_book("A Bend in the Road")

1984
Tom Clancy's Op-Center: Games of State (Tom Clancy's Op Center (Paperback))
Ground Zero and Beyond
Thousand Acres
Darkfall
The Cat Who Went into the Closet
